In [32]:
import locale
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
import seaborn as sns
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from zipfile import ZipFile

locale.setlocale(locale.LC_ALL, 'en_US')
sns.set()

EMISSIONS_DATA_URL = 'https://stats.oecd.org/Download.ashx?type=csv&Delimiter=%2c&IncludeTimeSeriesIdentifiers=False&LabelType=CodeAndLabel&LanguageCode=en'
GDP_DATA_URL = 'http://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=csv'

### Read data sets

In [23]:
# fetch data from stats.oecd.org
headers = {
    'Accept': 'text/html',
    'Referer': 'https://stats.oecd.org/modalexports.aspx?exporttype=bulk&FirstDataPointIndexPerPage=undefined&SubSessionId=ceea1636-28ef-4132-8376-2b008297b85c&Random=0.03143775516669467',
    'Cookie': 'ASP.NET_SessionId=ygiqojlmiuft3z1gafxeln1c',
}

res = requests.get(EMISSIONS_DATA_URL, headers=headers)

In [25]:
with StringIO(res.text) as infile:
    air_emissions = pd.read_csv(infile)

C:\Users\fredrik.fagerholm\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
gdp_world = pd.read_csv('../data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_10475113/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_10475113.csv', 
                        skiprows=4)

### Prepare GDP data

In [42]:
gdp_world['Indicator Name'].value_counts()

GDP (current US$)    264
Name: Indicator Name, dtype: int64

In [43]:
# data is in wide form: the GDP for a country in each row (years as columns) 
gdp_world.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.390503e+09,2.549721e+09,2.534637e+09,2.581564e+09,2.649721e+09,2.691620e+09,2.646927e+09,2.700559e+09,NaN,NaN
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.585657e+10,1.780429e+10,1.990732e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.904636e+10,1.954398e+10,NaN,NaN
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.379950e+10,1.117897e+11,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,NaN,NaN
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.192696e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322825e+10,1.138693e+10,1.188368e+10,1.303854e+10,NaN,NaN
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.012914e+09,NaN,NaN


In [44]:
gdp_world.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', 'Unnamed: 63'],
      dtype='object')

In [45]:
# to make the data easier to use we transform it to long form
# Country Name, Year and GDP as columns
gdp_long = gdp_world.melt(id_vars='Country Name', var_name='Year', value_name='GDP',
                          value_vars=[str(year) for year in range(1969, 2019)])
gdp_long['Year'] = gdp_long['Year'].astype(int)

# index data by Country name and year
gdp_country = gdp_long.sort_values(by=['Country Name', 'Year'])
gdp_country.set_index(['Country Name', 'Year'], inplace=True)

# create a pivot table with the year as index and country names as columns
gdp_pivot = gdp_long.pivot_table(values='GDP', index='Year', columns='Country Name')

In [46]:
# check for missing values and data types
pd.set_option("display.max_rows", 300)
pd.DataFrame({'Missing values': gdp_pivot.isnull().sum(), 
              'Data type': gdp_pivot.dtypes})

,Data type,Missing values
Country Name,,
Afghanistan,float64,21
Albania,float64,16
Algeria,float64,1
American Samoa,float64,34
Andorra,float64,2
Angola,float64,12
Antigua and Barbuda,float64,9
Arab World,float64,1
Argentina,float64,1


### Prepare emissions data

In [47]:
# the air emissions data contains the amount of different air pollutants emitted 
# for different countries and years
air_emissions.head()

,"ï»¿""COU""",Country,POL,Pollutant,VAR,Variable,YEA,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,SOX,Sulphur Oxides,TOT,Total man-made emissions,1990,1990,TONNE,Tonnes,3,Thousands,NaN,NaN,1585.754,NaN,NaN
1,AUS,Australia,SOX,Sulphur Oxides,TOT,Total man-made emissions,1991,1991,TONNE,Tonnes,3,Thousands,NaN,NaN,1570.777,NaN,NaN
2,AUS,Australia,SOX,Sulphur Oxides,TOT,Total man-made emissions,1992,1992,TONNE,Tonnes,3,Thousands,NaN,NaN,1652.946,NaN,NaN
3,AUS,Australia,SOX,Sulphur Oxides,TOT,Total man-made emissions,1993,1993,TONNE,Tonnes,3,Thousands,NaN,NaN,1743.161,NaN,NaN
4,AUS,Australia,SOX,Sulphur Oxides,TOT,Total man-made emissions,1994,1994,TONNE,Tonnes,3,Thousands,NaN,NaN,1764.906,NaN,NaN


In [48]:
# the variable indicates the meaning of the value in the Value column
# the emissions by emission source, or per capita
air_emissions.Variable.unique()

array(['Total man-made emissions', 'Total Mobile Sources',
       'Road Transport', 'Other Mobile Sources',
       'Total Stationary Sources', 'Power stations', 'Combustion',
       'Industrial combustion', 'Other combustion',
       'Industrial processes and product use', 'Agriculture',
       'Total emissions, Index 1990 = 100', 'Total emissions per capita',
       'Miscellaneous', 'Waste',
       'Total emissions per unit of GDP, Kg per 1000 USD',
       'Total emissions, Index 2000 = 100'], dtype=object)

In [49]:
# take rows with the total emissions for each country and year 
total_air_emissions = air_emissions[air_emissions['Variable'] == 'Total man-made emissions']
# take only interesting columns
total_air_emissions = total_air_emissions[['Country', 'Pollutant', 'Year', 'Value']]
# index data by country, pollutant and year and sort index
total_air_emissions = total_air_emissions.set_index(['Country', 'Pollutant', 'Year'])
total_air_emissions.sort_index(inplace=True)

### Visualize data

In [50]:
gdp_countries = gdp_country.index.levels[0]
emission_countries = total_air_emissions.index.levels[0]

common_country_names = np.intersect1d(gdp_countries, emission_countries)
pollutants = total_air_emissions.index.levels[1]

def plot_gdp(country, pollutant):
    try:
        gdp_per_year = gdp_country.loc[country]
        ax1 = gdp_per_year.plot(color='Green',
                                figsize=(16, 8))
        ax1.grid(False)
        ax1.set_title('GPD and {:s} emissions for {:s} (USD)'.format(pollutant, country), fontsize=16)
        ax1.set_ylabel('Billion dollars', fontsize=14)
        ax1.legend(loc='upper left')
        
        ax1.set_yticklabels(['${:.0f}BN'.format(val/1e9) for val in ax1.get_yticks()])
        
        ax2 = ax1.twinx()        
        
        emissions_per_year = total_air_emissions.loc[country, pollutant]
        ax2 = emissions_per_year.plot(ax=ax2)
        ax2.grid(False)
        ax2.legend([pollutant], loc='upper right')
        ax2.set_ylabel('Tonnes', fontsize=14)
        
        corrcoef = gdp_per_year.join(emissions_per_year).corr(method='pearson').iloc[0, 1]
        print("Pearson correlation coefficient: {:.4f}".format(corrcoef))
        
    except KeyError:
        print('No data on {:s} emissions for {:s}'.format(pollutant, country))
    
_ = interact(plot_gdp, country=common_country_names, pollutant=pollutants)

interactive(children=(Dropdown(description='country', options=('Australia', 'Austria', 'Belgium', 'Canada', 'Chile', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy', 'Japan', 'Latvia', 'Lithuania', 'Luxembourg', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland', 'Portugal', 'Slovak Republic', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'United Kingdom', 'United States'), value='Australia'), Dropdown(description='pollutant', options=('Carbon Monoxide', 'Nitrogen Oxides', 'Non-methane Volatile Organic Compounds', 'Particulates (PM10)', 'Particulates (PM2.5)', 'Sulphur Oxides'), value='Carbon Monoxide'), Output()), _dom_classes=('widget-interact',))

In [51]:
gdp_country.to_csv('../data/gdp_by_country_and_year.csv')
total_air_emissions.to_csv('../data/total_emissions_by_country_and_year.csv')